In [17]:
# Packages
import csv
import time
import datetime
import pandas as pd

In [25]:
# Function 01: every_10min_summary
def every_10min_summary(file):
    
    with open(file,'r') as csv_file:
        csv_reader = csv.reader(csv_file)

        next(csv_reader)
        list_shorted = []
        for line in csv_reader:
            if line[4] == "Contacts Shorted":
                list_shorted.append(line)

        # put EventTime to datetime format
        for line in list_shorted:

            line[0] = datetime.datetime.strptime(line[0], "%m/%d/%Y %H:%M")
            # Note: In documentation, it says the format of month must be zero-padded,
            # but it seems to work in this non-zero-padded scenario

        # divide into separate time slot
        stats = []
        temp = list_shorted[0][0]
        count = 0
        ten_minutes = datetime.timedelta(minutes=10)
        one_minutes = datetime.timedelta(minutes=1)
        one_hour = datetime.timedelta(hours=1)
        for line in list_shorted:
            if line == list_shorted[-1]:
                if temp - ten_minutes < line[0] <= temp:
                    count += 1
                    stats.append([temp - ten_minutes + one_minutes, temp, count])
                else:
                    stats.append([temp - ten_minutes + one_minutes, temp, count])

                    if line[0] < temp - ten_minutes:
                        temp = line[0]
                    else:
                        temp = temp - ten_minutes
                    count = 1
                    stats.append([temp - ten_minutes + one_minutes, temp, count])
            elif temp-ten_minutes < line[0] <= temp:

                count += 1
            else:

                stats.append([temp-ten_minutes+one_minutes, temp, count])

                if line[0] <= temp-ten_minutes:
                    temp = line[0]
                else:
                    temp = temp-ten_minutes
                count = 1

        # print out the result
        for line in stats:
            if line == ["Start Time","End Time", "Counts"]:
                pass
            line[0] = line[0].strftime("%d/%m/%Y %H:%M")
            line[1] = line[1].strftime("%d/%m/%Y %H:%M")

        result = pd.DataFrame(stats, columns=["Start Time", "End Time", "Counts"])

        return result

In [26]:
file = 'event_messages.csv'
every_10min_summary(file)

,Start Time,End Time,Counts
0,17/09/2020 19:08,17/09/2020 19:17,6
1,17/09/2020 18:57,17/09/2020 19:06,5
2,17/09/2020 18:47,17/09/2020 18:56,8
3,17/09/2020 18:34,17/09/2020 18:43,17
4,17/09/2020 18:23,17/09/2020 18:32,5
5,17/09/2020 18:13,17/09/2020 18:22,7
6,17/09/2020 18:01,17/09/2020 18:10,4
7,17/09/2020 17:44,17/09/2020 17:53,5
8,17/09/2020 17:22,17/09/2020 17:31,8
9,17/09/2020 15:57,17/09/2020 16:06,2


In [ ]:
# Function 02: hourly_summary
'''
From 17/09/2020 00:00 to 17/09/2020 24:00

Count the event "Contacts Shorted" in every one-hour slot, such as 00:00-01:00, 01:00-02:00 etc.

If no event counts in a certain hour, please count as zero, but make sure we have 24 hour slots
'''
def hourly_summary(file):
    